In [1]:
import re
import json

from concurrent.futures import ThreadPoolExecutor

import zhipuai


In [2]:
zhipuai.api_key = ""

In [3]:
classification_template = {
    "company": "",
    "year": [],
    "keyword": [],
    "formula": True,
    "type": ""
}

In [4]:
json.dumps(classification_template)

'{"company": "", "year": [], "keyword": [], "formula": true, "type": ""}'

In [5]:
def trim_string(s):
    return re.sub(r'^[ \'\"]*|[ \'\"]*$', '', s)

In [6]:
def get_llm_classification(question):
    assistant_role_msg = """你是一个提取问题句子要素清单的机器人，可以根据输入的问题句子，输出JSON格式的要素清单，模板：{"company": "", "year": [], "keyword": [], "formula": true, "type": ""}。
    其中company表示提取的公司名，year表示提取的年份，keyword表示提取的字段关键词，formula表示是否要使用公式。type字段表示问题类型，'type': '1'型问题为单字段的精准查询，'type': '1-2'型问题会涉及到两个或多个字段，'type': '2-1'型问题涉及到简单的公式计算，'type': '2-2'型问题涉及到查询多条记录，'type': '3-1'型问题是根据年报的具体文档段落做总结的开放式问题，'type': '3-2'型问题为金融领域专业知识问答。
    对于不确定的信息，你可以将输出空字符串或空列表。
    例如，给定问题"华翔股份2021年营业利润是多少元?"，应该输出:{"company": "华翔股份", "year": [2021], "keyword": ["营业利润"], "formula": false, "type": "1"}。
    给定问题"2021年利润总额最高的上市公司是？"，应该输出:{"company": "", "year": [2021], "keyword": ["利润总额"], "formula": false, "type": "1"}。
    给定问题"2019年，宁夏银星能源股份有限公司固定资产和无形资产分别是多少元?"，应该输出:{"company": "宁夏银星能源股份有限公司", "year": [2019], "keyword": ["固定资产", "无形资产"], "formula": false, "type": "1-2"}。
    给定问题"在北京注册的上市公司中，2019年资产总额最高的前四家上市公司是哪些家？金额为？"，应该输出:{"company": "", "year": [2019], "keyword": ["注册地址", "资产总额"], "formula": false, "type": "1-2"}。
    给定问题"2020年贵州燃气集团股份有限公司速动比率为多少?保留2位小数。"，应该输出:{"company": "贵州燃气集团股份有限公司", "year": [2020], "keyword": ["速动比率"], "formula": true, "type": "2-1"}。
    给定问题"2021年津药药业的法定代表人与上一年是否相同?"，应该输出:{"company": "津药药业", "year": [2020, 2021], "keyword": ["法定代表人"], "formula": false, "type": "2-2"}。
    给定问题"请简要分析爱丽家居科技股份有限公司2020年核心竞争力的情况。"，应该输出:{"company": "爱丽家居科技股份有限公司", "year": [2020], "keyword": ["核心竞争力"], "formula": false, "type": "3-1"}。
    给定问题"请简要介绍2021年久吾高科重大资产和股权出售情况。"，应该输出:{"company": "久吾高科", "year": [2021], "keyword": ["重大资产和股权出售"], "formula": false, "type": "3-1"}。
    给定问题"合同资产是指什么？"，应该输出:{"company": "", "year": [], "keyword": ["合同资产"], "formula": false, "type": "3-2"}。
    ……
    对于输出应该严格按照JSON格式输出，不要输出任何多余的字符。
    """
    response = zhipuai.model_api.invoke(
        model="chatglm_pro",
        prompt=[
            {"role": "user", "content": assistant_role_msg + f"\n那么，给定问题\"{question}\"，应该输出:"},
        ],
        ref={"enable": False}
    )

    result = response["data"]["choices"][0]["content"]
    result = trim_string(result).replace("\\", "")
    result = json.loads(result)
    return result

In [8]:
q = "康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？"
t = get_llm_classification(q)
d = {"question": q, "classification": t}

In [9]:
d

{'question': '康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？',
 'classification': {'company': '康希诺生物股份公司',
  'year': [2020],
  'keyword': ['资产负债比率'],
  'formula': True,
  'type': '1'}}

In [10]:
questions_file_list = [
    "./初赛/test_questions.json",
    "./复赛B/B-list-question.json"
]

In [11]:
def process_question(question):
    retry_count = 0
    classification = None
    while retry_count < 3:
        try:
            classification = get_llm_classification(question)
            break
        except Exception as _:
            retry_count += 1

    if classification:  # 如果分类成功，返回该条目
        return {"question": question, "classification": classification}
    return None

In [12]:
def extract_classification_from_file(file_name):
    with open(file_name, 'r', encoding="utf-8") as f:
        questions = [json.loads(line)["question"] for line in f.readlines()]

    # 使用 ThreadPoolExecutor 并行处理
    with ThreadPoolExecutor(max_workers=3) as executor:
        for item in executor.map(process_question, questions):
            if item:
                print(item)
                final_results.append(item)

In [ ]:
final_results = []
for file_path in questions_file_list:
    final_results.extend(extract_classification_from_file(file_path))

In [14]:
len(final_results)

179

In [15]:
# 如果需要将结果保存到文件中
with open("classified_questions.json", "w", encoding="utf-8") as f:
    json.dump(final_results, f, ensure_ascii=False, indent=4)
